In [1]:
# when installing the libraries after cloning the repository, use this command: pip install -r requirements.txt
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

import setuptools.dist

from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, LSTM
from keras.utils import pad_sequences

In [2]:
train_data = pd.read_csv("train.csv")

In [7]:
test_data = pd.read_csv("test.csv")

In [ ]:
def clean_data(df):
   
    clean_df = df.copy()

    clean_df .dropna(inplace=True)
    
    clean_df .drop_duplicates(inplace=True)
    
    clean_df ['cleaned_text'] = clean_df ['text'].apply(clean_text)
    

    return clean_df

#print(new_train_data.head())

In [ ]:
new_train_data = clean_data(train_data2)

In [20]:
import re
import unidecode

def clean_text(text):
    if pd.isna(text):
        return ""
    # Odstránenie URL
    text = re.sub(r'http\S+', '', text)
    # Odstránenie hashtagov
    text = re.sub(r'#\w+', '', text)
    # Odstránenie viacnásobných otáznikov a výkričníkov
    text = re.sub(r'(\?{2,}|\!{2,})', '', text)
    # Odstránenie špeciálnych znakov (ponechanie len alfanumerických znakov a medzier)
    text = re.sub(r'[^\w\s]', '', text)
    # Normalizácia textu - odstránenie diakritiky
    text = unidecode.unidecode(text)
    # Previesť text na malé písmená
    text = text.lower()
    return text

def remove_numbers(text):
    return re.sub(r'\d+', '', text)

def clean_data(df):
  
    clean_df = df.copy()
    clean_df.drop(columns=['location'], inplace=True)
    clean_df['keyword'].fillna('unknown', inplace=True)
    clean_df['keyword'] = clean_df['keyword'].apply(clean_text)
    clean_df['keyword'] = clean_df['keyword'].apply(remove_numbers)
    clean_df['text'] = clean_df['text'].apply(clean_text)

    return clean_df


In [21]:
new_train_data = clean_data(train_data)

new_train_data.head(20)

,id,keyword,text,target
0,1,unknown,our deeds are the reason of this may allah fo...,1
1,4,unknown,forest fire near la ronge sask canada,1
2,5,unknown,all residents asked to shelter in place are be...,1
3,6,unknown,13000 people receive evacuation orders in cal...,1
4,7,unknown,just got sent this photo from ruby as smoke f...,1
5,8,unknown,update california hwy 20 closed in both dire...,1
6,10,unknown,heavy rain causes flash flooding of streets ...,1
7,13,unknown,im on top of the hill and i can see a fire in ...,1
8,14,unknown,theres an emergency evacuation happening now i...,1
9,15,unknown,im afraid that the tornado is coming to our area,1
